In [ ]:
import pandas as pd
import numpy as np
import glob
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# ============================
# 1. Cargar todos los portales
# ============================

files = glob.glob("INPUT_GLOB/Punto3_*_CARPETA.xlsx")   # <-- cambia SEGUN NOMBRE DE LA CARPETA DONDE ESTEN LOS ARCHIVOS RESULTADO PUNTO 3

print("Archivos encontrados:", len(files))
for f in files:
    print(" -", f)

df = pd.concat([pd.read_excel(f) for f in files], ignore_index=True)

# ============================
# 2. Asegurar columnas clave
# ============================

df = df.dropna(subset=["portal_maturity_level"])
df["portal_maturity_level"] = df["portal_maturity_level"].str.lower().str.strip()

# ============================
# 3. Definir métricas
# ============================

METRICS = {
    "Índice Global": "portal_maturity",
    "Trazabilidad Global": "traceability_score",
    "Interoperabilidad Técnica": "interoperability_technical",
    "Interoperabilidad Semántica": "interoperability_semantics",
    "Accesibilidad": "accessibility_score",
    "Calidad Metadatos": "quality_score"
}

results = []
tukey_results = []

# ============================
# 4. ANOVA + Kruskal + Tukey
# ============================

for label, col in METRICS.items():

    if col not in df.columns:
        print(f"⚠️ La columna {col} no existe en los datos.")
        continue

    g = df[[col, "portal_maturity_level"]].dropna()

    groups = [
        g[g["portal_maturity_level"] == lvl][col].values
        for lvl in ["bajo", "medio", "alto"]
        if (g["portal_maturity_level"] == lvl).sum() >= 3
    ]

    if len(groups) < 2:
        print(f"⚠️ No hay suficientes grupos para ANOVA en {label}")
        continue

    # ANOVA
    F, p_anova = stats.f_oneway(*groups)

    # Kruskal
    H, p_kw = stats.kruskal(*groups)

    # Guardar resultados principales
    results.append({
        "Métrica": label,
        "ANOVA_p": p_anova,
        "Kruskal_p": p_kw,
        "Media_bajo": g[g["portal_maturity_level"]=="bajo"][col].mean(),
        "Media_medio": g[g["portal_maturity_level"]=="medio"][col].mean(),
        "Media_alto": g[g["portal_maturity_level"]=="alto"][col].mean()
    })

    # Tukey si ANOVA es significativo
    if p_anova < 0.05:
        tukey = pairwise_tukeyhsd(
            endog=g[col],
            groups=g["portal_maturity_level"],
            alpha=0.05
        )

        tukey_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])
        tukey_df["Métrica"] = label
        tukey_results.append(tukey_df)

# ============================
# 5. Exportar a Excel
# ============================

with pd.ExcelWriter("Punto5_Resultados_Estadisticos2.xlsx") as writer:
    pd.DataFrame(results).sort_values("ANOVA_p").to_excel(writer, sheet_name="ANOVA_Kruskal", index=False)

    if tukey_results:
        pd.concat(tukey_results, ignore_index=True).to_excel(writer, sheet_name="Tukey", index=False)

print("\n Archivo generado: Punto5_Resultados_Estadisticos.xlsx")    # <-- cambia SEGUN  CRITERIO DEL USUARIO